In [1]:
import torch
from torch import nn
from model import TransformerDiffusionModel
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
train_data_path = "../../train_daily.csv"
test_data_path = "../../test_daily.csv"

In [3]:
input_days = 90       # 输入序列长度（90天）
output_days = 365

use_cols = [
    'Global_active_power','Global_reactive_power',
    'Global_intensity','Sub_metering_1',
    'Sub_metering_2','Sub_metering_3',
    'sub_metering_remainder'
]

def load_daily(path):
    df = pd.read_csv(path, parse_dates=['DateTime'])
    df.sort_values('DateTime', inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df[['DateTime'] + use_cols].copy()
    df.ffill(inplace=True)
    df.bfill(inplace=True)
    return df

train_df = load_daily(train_data_path)
test_df  = load_daily(test_data_path)

full_df = pd.concat([train_df, test_df], ignore_index=True)

scaled = full_df[use_cols].values  # 直接使用原始值
target = scaled[:, 0]

def build_samples(data, target, input_len, output_len, step=1):
    X, y = [], []
    for i in range(input_len, len(data) - output_len + 1, step):
        X.append(data[i - input_len:i])
        y.append(target[i:i + output_len])
    return np.array(X), np.array(y)

X_all, y_all = build_samples(scaled, target, input_days, output_days, step=1)

split_index = len(train_df) - input_days
X_train = X_all[:split_index]
y_train = y_all[:split_index]
X_test  = X_all[split_index:]
y_test  = y_all[split_index:]

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test  shape: {X_test.shape}")
print(f"y_test  shape: {y_test.shape}")


X_train shape: (657, 90, 7)
y_train shape: (657, 365)
X_test  shape: (331, 90, 7)
y_test  shape: (331, 365)


In [4]:
from torch.utils.data import TensorDataset, DataLoader, random_split
import matplotlib.pyplot as plt
import os

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
batch_size = 16

train_ds = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(y_train, dtype=torch.float32)
)
val_size = int(len(train_ds) * 0.2)
train_size = len(train_ds) - val_size
train_subset, val_subset = random_split(train_ds, [train_size, val_size])

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

In [5]:
os.makedirs("output/models/pred365", exist_ok=True)
os.makedirs("output/pics/loss/pred365", exist_ok=True)

# 保存所有 loss 用于计算平均曲线
all_train_losses = []
all_val_losses = []
mae_list = []
mse_list = []

for run in range(1, 6):
    print(f"\n=== Run {run} ===")


    # model 
    model = TransformerDiffusionModel(
        input_dim=7,          # 输入特征维度
        d_model=256,          # Transformer 隐藏维度
        nhead=8,              # 注意力头数
        num_layers=3,         # Transformer 层数
        output_seq_len=365,    # 预测输出长度
        dropout=0.1,          # Dropout 比例
        input_seq_len=90      # 输入序列长度
    ).to(device)


    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

    train_losses, val_losses = [], []
    best_val = float('inf')
    best_model = None
    best_mae = None
    best_mse = None
    patience = 10
    no_improve = 0

    for epoch in range(1, 101):
        model.train()
        total_train_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item() * xb.size(0)
        avg_train = total_train_loss / len(train_subset)
        train_losses.append(avg_train)

        # 验证
        all_preds, all_gts = [], []
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss = criterion(pred, yb)
                total_val_loss += loss.item() * xb.size(0)
                all_preds.append(pred.cpu().numpy())
                all_gts.append(yb.cpu().numpy())


        all_preds = np.concatenate(all_preds, axis=0)
        all_gts = np.concatenate(all_gts, axis=0)

        avg_val = total_val_loss / len(val_subset)
        val_losses.append(avg_val)

        val_mae = mean_absolute_error(all_gts, all_preds)
        val_mse = mean_squared_error(all_gts, all_preds)
        if epoch % 10 ==0:
            print(f"Run {run} | Epoch {epoch:03d} | Train Loss: {avg_train:.6f} | "
            f"Val Loss: {avg_val:.6f} | MAE: {val_mae:.4f} | MSE: {val_mse:.4f}")

        # 保存最优模型
        if avg_val < best_val:
            best_val = avg_val
            best_model = model.state_dict()
            best_mae = val_mae
            best_mse = val_mse
            torch.save(best_model, f"output/models/pred365/run_{run}.pth")
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

    # 保存单组loss图
    plt.figure(figsize=(8, 5))
    plt.plot(train_losses, label="Train MSE")
    plt.plot(val_losses, label="Val MSE")
    plt.xlabel("Epoch")
    plt.ylabel("Loss (MSE)")
    plt.title(f"Loss Curve - Run {run}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"output/pics/loss/pred365/run_{run}.png", dpi=300)
    plt.close()

    # 收集供后续平均绘图
    all_train_losses.append(train_losses)
    all_val_losses.append(val_losses)
    mae_list.append(best_mae)
    mse_list.append(best_mse)

    
print("\n=== Final Evaluation over 5 Runs ===")
print(f"MAE: mean = {np.mean(mae_list):.4f}, std = {np.std(mae_list):.4f}")
print(f"MSE: mean = {np.mean(mse_list):.4f}, std = {np.std(mse_list):.4f}")





=== Run 1 ===
Run 1 | Epoch 010 | Train Loss: 0.086321 | Val Loss: 0.104044 | MAE: 0.2496 | MSE: 0.1040
Run 1 | Epoch 020 | Train Loss: 0.048161 | Val Loss: 0.112358 | MAE: 0.2629 | MSE: 0.1124
Early stopping at epoch 22

=== Run 2 ===
Run 2 | Epoch 010 | Train Loss: 0.085510 | Val Loss: 0.101138 | MAE: 0.2463 | MSE: 0.1011
Run 2 | Epoch 020 | Train Loss: 0.048265 | Val Loss: 0.108214 | MAE: 0.2579 | MSE: 0.1082
Early stopping at epoch 22

=== Run 3 ===
Run 3 | Epoch 010 | Train Loss: 0.088453 | Val Loss: 0.101685 | MAE: 0.2472 | MSE: 0.1017
Run 3 | Epoch 020 | Train Loss: 0.050910 | Val Loss: 0.108550 | MAE: 0.2579 | MSE: 0.1085
Early stopping at epoch 23

=== Run 4 ===
Run 4 | Epoch 010 | Train Loss: 0.085997 | Val Loss: 0.100974 | MAE: 0.2467 | MSE: 0.1010
Run 4 | Epoch 020 | Train Loss: 0.048834 | Val Loss: 0.107658 | MAE: 0.2570 | MSE: 0.1077
Early stopping at epoch 22

=== Run 5 ===
Run 5 | Epoch 010 | Train Loss: 0.087570 | Val Loss: 0.101564 | MAE: 0.2474 | MSE: 0.1016
Run 5 |

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import torch

os.makedirs("output/pics/comparison/pred365", exist_ok=True)

gts_day = y_test

# 缓存所有模型预测
all_model_preds_day = []

for run in range(1, 6):
    print(f"\n=== Run {run} Evaluation ===")

    # 加载模型
    model = TransformerDiffusionModel(
        input_dim=7,          # 输入特征维度
        d_model=256,          # Transformer 隐藏维度
        nhead=8,              # 注意力头数
        num_layers=3,         # Transformer 层数
        output_seq_len=365,    # 预测输出长度
        dropout=0.1,          # Dropout 比例
        input_seq_len=90      # 输入序列长度
    ).to(device)
    model.load_state_dict(torch.load(f"output/models/pred365/run_{run}.pth"))
    model.eval()

    # 推理
    test_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32)
    )
    test_loader = torch.utils.data.DataLoader(test_ds, batch_size=256, shuffle=False)

    preds = []
    with torch.no_grad():
        for xb, _ in test_loader:
            xb = xb.to(device)
            pred = model(xb).cpu().numpy()
            preds.append(pred)
    preds = np.concatenate(preds, axis=0)  
    preds_day = preds  

    all_model_preds_day.append(preds_day)  # 保留每组 day 级预测

    # ==== 多样本评估 ====
    sample_ids = [5 * (run - 1) + i for i in range(5)]  # 每组画 5 个样本
    mae_list = []
    mse_list = []

    for idx in sample_ids:
        gt = gts_day[idx]
        pred = preds_day[idx]
        mae = mean_absolute_error(gt, pred)
        mse = mean_squared_error(gt, pred)
        mae_list.append(mae)
        mse_list.append(mse)

        # 图保存
        plt.figure(figsize=(10, 4.5))
        plt.plot(gt, label="Ground Truth", linewidth=2)
        plt.plot(pred, label="Prediction", linewidth=2)
        plt.xlabel("Day")
        plt.ylabel("Avg Power (kW)")
        plt.title(f"Run {run} - Sample {idx} | MAE: {mae:.4f}, MSE: {mse:.4f}")
        plt.xticks(np.arange(0, 366, 20), labels=[str(i+1) for i in range(0, 366, 20)])
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"output/pics/comparison/pred365/run_{run}_sample_{idx}.png", dpi=300)
        plt.close()

    # 打印指标
    print(f"[Run {run}] MAE (day): mean = {np.mean(mae_list):.4f}, std = {np.std(mae_list):.4f}")
    print(f"[Run {run}] MSE (day): mean = {np.mean(mse_list):.4f}, std = {np.std(mse_list):.4f}")

# ==== 平均预测绘图 ====
print("\n=== Averaged Prediction over 5 Models ===")
for i in range(5):  # 样本编号：0~4
    idx = i * 5
    gt = gts_day[idx]

    preds_across_models = [preds_day[idx] for preds_day in all_model_preds_day]
    avg_pred = np.mean(preds_across_models, axis=0)

    mae = mean_absolute_error(gt, avg_pred)
    mse = mean_squared_error(gt, avg_pred)

    plt.figure(figsize=(10, 4.5))
    plt.plot(gt, label="Ground Truth", linewidth=2)
    plt.plot(avg_pred, label="Avg Prediction", linewidth=2)
    plt.xlabel("Day")
    plt.ylabel("Avg Power (kW)")
    plt.title(f"Averaged Prediction - Sample {idx} | MAE: {mae:.4f}, MSE: {mse:.4f}")
    plt.xticks(np.arange(0, 366, 20), labels=[str(i+1) for i in range(0, 366, 20)])
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"output/pics/comparison/pred365/avg_sample_{idx}.png", dpi=300)
    plt.close()



=== Run 1 Evaluation ===


/tmp/ipykernel_557543/1155871921.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"output/models/pred365/run_{run}.pth"))


[Run 1] MAE (day): mean = 0.2308, std = 0.0048
[Run 1] MSE (day): mean = 0.0860, std = 0.0024

=== Run 2 Evaluation ===


/tmp/ipykernel_557543/1155871921.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"output/models/pred365/run_{run}.pth"))


[Run 2] MAE (day): mean = 0.2373, std = 0.0085
[Run 2] MSE (day): mean = 0.0920, std = 0.0050

=== Run 3 Evaluation ===


...

[Run 3] MAE (day): mean = 0.2467, std = 0.0049
[Run 3] MSE (day): mean = 0.0958, std = 0.0033

=== Run 4 Evaluation ===


/tmp/ipykernel_557543/1155871921.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"output/models/pred365/run_{run}.pth"))


[Run 4] MAE (day): mean = 0.2564, std = 0.0065
[Run 4] MSE (day): mean = 0.0995, std = 0.0043

=== Run 5 Evaluation ===


/tmp/ipykernel_557543/1155871921.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"output/models/pred365/run_{run}.pth"))


[Run 5] MAE (day): mean = 0.2331, std = 0.0056
[Run 5] MSE (day): mean = 0.0864, std = 0.0031

=== Averaged Prediction over 5 Models ===
